# Jリーグ公式戦2014年シーズン後半戦全試合の観客動員数を予測するモデルの作成


1. Step 1: データの準備
2. Step 2: ひとまず可視化
3. Step 3: scikit-learnを使った重回帰分析
4. Step 4: 学習（Training）と検証（Validation）

In [12]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
sns.set_style('whitegrid')
%matplotlib inline

In [13]:
train_data_df = pd.read_csv('./data/train.csv')

In [14]:
print(train_data_df.head())

      id      y  year stage   match   gameday   time       home      away  \
0  13994  18250  2012    Ｊ１  第１節第１日  03/10(土)  14:04     ベガルタ仙台  鹿島アントラーズ   
1  13995  24316  2012    Ｊ１  第１節第１日  03/10(土)  14:04   名古屋グランパス   清水エスパルス   
2  13996  17066  2012    Ｊ１  第１節第１日  03/10(土)  14:04      ガンバ大阪   ヴィッセル神戸   
3  13997  29603  2012    Ｊ１  第１節第１日  03/10(土)  14:06  サンフレッチェ広島     浦和レッズ   
4  13998  25353  2012    Ｊ１  第１節第１日  03/10(土)  14:04   コンサドーレ札幌    ジュビロ磐田   

        stadium                                  tv  
0  ユアテックスタジアム仙台                 スカパー／ｅ２／スカパー光／ＮＨＫ総合  
1       豊田スタジアム    スカパー／ｅ２／スカパー光（Ｊ　ＳＰＯＲＴＳ　４）／ＮＨＫ名古屋  
2       万博記念競技場     スカパー／ｅ２／スカパー光（Ｊ　ＳＰＯＲＴＳ　１）／ＮＨＫ大阪  
3  エディオンスタジアム広島                 スカパー／ｅ２／スカパー光／ＮＨＫ広島  
4         札幌ドーム  スカパー／ｅ２／スカパー光（スカイ・Ａ　ｓｐｏｒｔｓ＋）／ＮＨＫ札幌  


In [43]:
#文字列を数値に変換するマッピング&説明変数
trans_data = train_data_df[['stage','match','gameday','time','home','away','stadium','tv']]
drop_cols = ['id','y']
X_multi = train_data_df.drop(drop_cols,1)
for column in trans_data.columns.values:
    unique_data = trans_data[column].unique()
    data_mapping = {data:idx for  idx, data in enumerate(unique_data)}
    X_multi[column] = trans_data[column].map(data_mapping)

print(X_multi.head())

   year  stage  match  gameday  time  home  away  stadium  tv
0  2012      0      0        0     0     0     0        0   0
1  2012      0      0        0     0     1     1        1   1
2  2012      0      0        0     0     2     2        2   2
3  2012      0      0        0     1     3     3        3   3
4  2012      0      0        0     0     4     4        4   4


In [44]:
# 目的変数
Y_target = train_data_df.y
print(Y_target.shape)

(1721,)


In [45]:
# モデルを作ります。
lreg = LinearRegression()
lreg.fit(X_multi,Y_target)

/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [47]:
# 新しいDataFrameを作ります。
coeff_df = DataFrame(X_multi.columns)
print(coeff_df)
coeff_df.columns = ['Features']

#求められた係数を代入します。
coeff_df["Coefficient Estimate"] = pd.Series(lreg.coef_)

coeff_df

         0
0     year
1    stage
2    match
3  gameday
4     time
5     home
6     away
7  stadium
8       tv


,Features,Coefficient Estimate
0,year,1681.708387
1,stage,-7698.354504
2,match,59.121694
3,gameday,-17.580858
4,time,38.013900
5,home,61.947939
6,away,-81.797908
7,stadium,-145.176874
8,tv,9.427585
